In [ ]:
import os
import pandas as pd

# Define board coordinates
board_size = 19
coordinates = [chr(97 + row) + chr(97 + col) for row in range(board_size) for col in range(board_size)]

# Define columns for the final DataFrame
columns = ["game_id", "move_id", "color", "our_players_color", "winner_color", "winner_score", "result", "rules", "handicap", "starter_player", "step_count"] + coordinates

# Initialize an empty DataFrame
all_games_df = pd.DataFrame(columns=columns)

# Helper functions
def parse_metadata_and_moves(sgf_content):
    moves = []
    # Determine rules and handicap
    rules = "Unknown"
    handicap = 0

    if "RU[" in sgf_content:
        rules = sgf_content.split("RU[")[1].split("]")[0]
    if "HA[" in sgf_content:
        handicap = int(sgf_content.split("HA[")[1].split("]")[0])
    
    # Determine winner information
    winner_color = "black" if "RE[B+" in sgf_content else "white"
    result = "resign" if "Resign" in sgf_content else "score"
    winner_score = None
    if result == "score":
        winner_score = sgf_content.split("RE[")[1].split("]")[0][2:]
    
    # Extract moves
    sgf_moves = sgf_content.split(";")[2:]  # Skip the first two header parts
    move_id = 1
    for move in sgf_moves:
        color = "black" if move.startswith("B") else "white"
        pos = move[2:4]
        moves.append((move_id, color, pos))
        move_id += 1
    
    # Determine starter player based on the first move
    starter_player = moves[0][1] if moves else "unknown"  # "unknown" if there are no moves
    return moves, winner_color, winner_score, result, rules, handicap, starter_player

# Initialize board with all 0s
def initialize_board():
    return {coord: 0 for coord in coordinates}

# Apply moves and log each step for a single game
def process_game(game_id, moves, winner_color, winner_score, result, our_players_color, rules, handicap, starter_player):
    board_state = initialize_board()
    data = []
    step_count = len(moves)

    for move_id, color, pos in moves:
        # Update board with current move
        board_state[pos] = 2 if color == "black" else 1  # 2 for Black, 1 for White
        row_data = {
            "game_id": game_id,
            "move_id": move_id,
            "color": color,
            "our_players_color": our_players_color,
            "winner_color": winner_color,
            "winner_score": winner_score,
            "result": result,
            "rules": rules,
            "handicap": handicap,
            "starter_player": starter_player,
            "step_count": step_count
        }
        row_data.update(board_state)
        data.append(row_data.copy())
        # Reset the stone to 0 for the next move
        board_state[pos] = 0
    return data

# Parse and process multiple SGF files in a folder
def process_sgf_folder(folder_path, our_players_color):
    global all_games_df
    game_id = 1
    for filename in os.listdir(folder_path):
        if filename.endswith(".sgf"):
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                sgf_content = file.read()
            moves, winner_color, winner_score, result, rules, handicap, starter_player = parse_metadata_and_moves(sgf_content)
            game_data = process_game(game_id, moves, winner_color, winner_score, result, our_players_color, rules, handicap, starter_player)
            game_df = pd.DataFrame(game_data, columns=columns)
            all_games_df = pd.concat([all_games_df, game_df], ignore_index=True)
            game_id += 1

# Specify the folder path and the player's color
folder_path = "SGF"  # Update with the path to your folder
our_players_color = "black"  # Or "white", depending on the player  # HERE MAYBE WE SHOULD USE A KEY TABLE THAT WE WIL BUILD, AND JOIN THE 2 TABLES BY A KEY ID--------------------------------------------------------

# Process the SGF files
process_sgf_folder(folder_path, our_players_color)

# Save the merged DataFrame to an Excel file
output_path = "merged_games_data_new.xlsx"
all_games_df.to_excel(output_path, index=False)

print(f"Data saved to {output_path}")
